In [17]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import time
import os

In [18]:
#extract data
# this part of code is taken from tensorflow tutorial
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
print(len(mnist.validation.labels))
trX, trY,vdX,vdY, teX, teY = mnist.train.images, mnist.train.labels,mnist.validation.images, mnist.validation.labels, mnist.test.images, mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
5000


In [19]:
def clamp(s):
    return tf.clip_by_value(s, 0.,1.)
# Glorot/Bengio weight initialization
# this function is copied from Benjamin Scelliers code
def initialize_layer(n_in, n_out):
    rng = np.random.RandomState()
    W_values = np.asarray(
        rng.uniform(
            low=-np.sqrt(6. / (n_in + n_out)),
            high=np.sqrt(6. / (n_in + n_out)),
            size=(n_in, n_out)
        ),
        dtype=np.float32
    )
    return W_values

def rho(s):
    return tf.sigmoid(s)
def clampByNumpy(s):
    return np.clip(s,0.,1.)
def rhoByNumpy(s):
    return 1./(1+np.exp(-s))
def save_params(path, biases, weights, training_curves):
        f = file(path, 'wb')
        to_dump        = biases, weights, training_curves
        cPickle.dump(to_dump, f, protocol=cPickle.HIGHEST_PROTOCOL)
        f.close()

In [20]:
def free_phase_energy(inputLayer, hiddenLayer, outputLayer, hiddenWeight, outputWeight, inputBias, hiddenBias, outputBias):
    squared_term= tf.reduce_sum(tf.square(inputLayer))+tf.reduce_sum(tf.square(hiddenLayer))+tf.reduce_sum(tf.square(outputLayer))
    biased_term= tf.reduce_sum(tf.multiply(inputLayer, inputBias))+tf.reduce_sum(tf.multiply(hiddenLayer, hiddenBias))+tf.reduce_sum(tf.multiply(outputLayer, outputBias))
    weighted_term1= tf.reduce_sum(tf.multiply(tf.matmul(tf.reshape(inputLayer,[1,784]),hiddenWeight),rho(hiddenLayer)))
    weighted_term2=tf.reduce_sum(tf.multiply(tf.matmul(tf.reshape(rho(hiddenLayer),[1,500]), outputWeight),rho(outputLayer)))
    weighted_term= weighted_term1+weighted_term2
    return squared_term/2.- weighted_term/2. - biased_term
    

In [21]:
def weakly_clamped_energy(inputLyer, hiddenLayer, outputLayer, hiddenWeight, outputWeight, inputBias, hiddenBias, outputBias, beta, yData):
    #implement F= E + beta * C
    # C= ||prediction - yData||^2
    E= free_phase_energy(inputLyer, hiddenLayer, outputLayer, hiddenWeight, outputWeight, inputBias, hiddenBias, outputBias)
    C= tf.reduce_sum(tf.square(tf.subtract(yData,outputLayer)))
    F= tf.add(E, tf.multiply(beta,C))
    return F

In [25]:
n_epochs=10
alphas=[0.5,0.05]
bt=0.001
dir_path = os.path.dirname(os.path.abspath("__file__"))
path= dir_path+os.sep+"net1.net"
layer_sizes = [28*28] + [500] + [10]
biases_values  = [np.zeros((size), dtype=np.float32) for size in layer_sizes]
weights_values = [initialize_layer(size_pre,size_post) for size_pre,size_post in zip(layer_sizes[:-1],layer_sizes[1:])]
training_curves = dict()
training_curves["training error"]   = list()
training_curves["validation error"] = list()
#create tensors
H_var= tf.Variable(np.zeros(500, dtype=np.float32),name="hidden_layer_variable")
O_var= tf.Variable(np.zeros(10, dtype=np.float32), name="output_layer_variable")
Hw_var= tf.Variable(weights_values[0], name="hidden_weight_variable")
Ow_var= tf.Variable(weights_values[1], name="output_weight_variable")
Bin_var= tf.Variable(biases_values[0], name="input_bias_variable")
Bh_var= tf.Variable(biases_values[1], name="hidden_bias_variable")
Bo_var= tf.Variable(biases_values[2], name="output_bias_variable")
# create placeholder
H_ph= tf.placeholder(dtype=tf.float32, shape=[500])
O_ph= tf.placeholder(dtype=tf.float32, shape=[10])
Hw_ph=tf.placeholder(dtype=tf.float32, shape=weights_values[0].shape)
Ow_ph=tf.placeholder(dtype=tf.float32, shape=weights_values[1].shape)
Bin_ph= tf.placeholder(dtype=tf.float32, shape=biases_values[0].shape)
Bh_ph= tf.placeholder(dtype=tf.float32, shape=biases_values[1].shape)
Bo_ph= tf.placeholder(dtype=tf.float32, shape=biases_values[2].shape)
beta= tf.placeholder(dtype=tf.float32)
y_data= tf.placeholder(dtype=tf.float32, shape=[10])

IL= tf.placeholder(dtype=tf.float32, shape=[784])

energy_of_free_phase= free_phase_energy(IL, H_var, O_var,Hw_ph, Ow_ph, Bin_ph, Bh_ph, Bo_ph)
totalEnergy= weakly_clamped_energy(IL, H_var, O_var,Hw_ph, Ow_ph, Bin_ph, Bh_ph, Bo_ph, beta, y_data)
free_phase_optimization=tf.train.GradientDescentOptimizer(0.01).minimize(energy_of_free_phase)
weakly_clamped_optimization= tf.train.GradientDescentOptimizer(0.01).minimize(totalEnergy)

count=0
start_time = time.clock()
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(n_epochs):
        print("starting epoch "+ str(i+1))
        error=0.
        for x,y in zip(trX,trY):
            #clamp each input
            cx=sess.run(clamp(x))
            count =count +1
            if count==100:
                print("100th Milestone")
                count=0
            #print("clamping done")
            #create layer
            inputLayer= x
            hiddenLayer= np.zeros(500, dtype=np.float32)
            outputLayer= np.zeros(10, dtype=np.float32)
            #run free phase
            
            sess.run(free_phase_optimization, feed_dict={IL:cx, Hw_ph:weights_values[0], Ow_ph:weights_values[1], Bin_ph:biases_values[0], Bh_ph:biases_values[1], Bo_ph:biases_values[2]})
            # save hidden layer and output layer values
            free_hidden_layer= sess.run(H_var)
            free_output_layer= sess.run(O_var)
            #print(free_output_layer)
            sign = 2*np.random.randint(0,2)-1 # random sign +1 or -1
            bt = np.float32(sign*bt)
            #run weakly clamped phase
            sess.run(weakly_clamped_optimization, feed_dict={IL:cx, Hw_ph:weights_values[0], Ow_ph:weights_values[1], Bin_ph:biases_values[0], Bh_ph:biases_values[1], Bo_ph:biases_values[2], beta:bt, y_data:y})
            weakly_clamped_hidden_layer=sess.run(H_var)
            weakly_clamped_output_layer=sess.run(O_var)
            
            #now implement the update bias and beta 
            partA= np.matmul(rhoByNumpy(cx).reshape(len(cx),1), rhoByNumpy(weakly_clamped_hidden_layer).reshape(1, len(weakly_clamped_hidden_layer)))
            partB= np.matmul(rhoByNumpy(cx).reshape(len(cx),1), rhoByNumpy(free_hidden_layer).reshape(1, len(free_hidden_layer)))
            delW0=-1*(partA-partB)/2.
            
            partC= np.matmul(rhoByNumpy(weakly_clamped_hidden_layer).reshape(len(weakly_clamped_hidden_layer),1), rhoByNumpy(weakly_clamped_output_layer).reshape(1, len(weakly_clamped_output_layer)))
            partD= np.matmul(rhoByNumpy(free_hidden_layer).reshape(len(free_hidden_layer),1), rhoByNumpy(free_output_layer).reshape(1, len(free_output_layer)))
            delW1= -1*(partC-partD)/2.
            #update Weights
            weights_values[0]=weights_values[0]- delW0*alphas[0]
            weights_values[1]=weights_values[1]- delW1*alphas[1]
            
            #get delB
            delB1= -1*(rhoByNumpy(weakly_clamped_hidden_layer)-rhoByNumpy(free_hidden_layer))
            delB2=-1*(rhoByNumpy(weakly_clamped_output_layer)-rhoByNumpy(free_output_layer))
            #update bias
            biases_values[1]=biases_values[1]- alphas[0]*delB1
            biases_values[2]=biases_values[2]- alphas[1]*delB2
            
            error=error+ (np.argmax(y)==np.argmax(free_output_layer))*1.
            #print("image processed")
        training_curves["training error"].append(np.average(error))
        print("Epoch "+str(i+1)+" training error rate: "+ str(np.average(error)))
        #check validation set
        #run free phase with validation data
        verror=0.
        for x,y in zip(vdX, vdY):
            cx=sess.run(clamp(x))
            hiddenLayer= np.zeros(500, dtype=np.float32)
            outputLayer= np.zeros(10, dtype=np.float32)
            sess.run(free_phase_optimization, feed_dict={IL:cx, Hw_ph:weights_values[0], Ow_ph:weights_values[1], Bin_ph:biases_values[0], Bh_ph:biases_values[1], Bo_ph:biases_values[2]})
            pred= sess.run(O_var)
            verror= verror+ (np.argmax(y)==np.argmax(pred))*1.
        #test error
        training_curves["validation error"].append(np.average(verror))
        print("Epoch "+str(i+1)+" validation error rate: "+ str(np.average(verror)))
        
            # in free phase layer will be a variable and weights and bias will be fixed
    
    save_params(path, biases_values, weights_values, training_curves)
duration = (time.clock() - start_time) / 60.
print("   duration=%.1f min" % (duration))

starting epoch 1
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th Milestone
100th M

KeyboardInterrupt: 